In [ ]:
## Importing packages

import pandas as pd
import numpy as np
import itertools

import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.arima.model import ARIMA

In [ ]:
## Reading data
dataRaw = pd.read_csv('./prices250.txt', delimiter = '\s+', header = None).T

## Adding column names (for each timepoint)
dataRaw.columns = ['price' + str(x) for x in range(1, dataRaw.shape[1]+1, 1)]

## Adding column representing which instrument
dataRaw['instrument'] = ['instrument' + str(x) for x in range(1, dataRaw.shape[0]+1, 1)]

## Converting to long format
data = pd.wide_to_long(dataRaw, stubnames = 'price', i = 'instrument', j = 'time')
data['instrument'] = [x[0] for x in data.index]
data['time'] = [x[1] for x in data.index]
data.reset_index(drop = True, inplace = True)

In [ ]:
## Autocorrelation graph

for i in range(1, 101, 1):
    instrument = data[data['instrument'] == ('instrument' + str(i))]['price']
    pd.plotting.autocorrelation_plot(instrument, linewidth = 0.5)

## Instrument 1 ARIMA

In [ ]:
## Subset for testing

dataInstrument1 = data[data["instrument"] == "instrument1"][["price", "time"]].set_index("time")
dataInstrument1.index = pd.to_datetime(dataInstrument1.index, unit = 'd')

In [ ]:
## Looking for optimal ARIMA parameters

# trainData = dataInstrument1["price"][:225]
# testData = dataInstrument1["price"][225:]

# p = range(30, 31, 1)
# d = q = range(0, 2)
# pdq = list(itertools.product(p, d, q))
# modelAIC = []

# for paramlist in pdq:
#     arimaModel = ARIMA(trainData, order = paramlist)
#     arimaModelFit = arimaModel.fit()
#     modelAIC.append(arimaModelFit.aic)

# for i in range(len(modelAIC)):
#     if list(modelAIC == min(modelAIC))[i] == True:
#         print(pdq[i])

In [ ]:
## Training using optimised parameters

trainData = dataInstrument1["price"][:225]
testData = dataInstrument1["price"][225:]
arimaModel = ARIMA(trainData, order = (30, 1, 0))
arimaModelFit = arimaModel.fit()
predictions = arimaModelFit.forecast(steps = 25)

testData.plot()
predictions.plot(color = "red")

In [400]:
def shortTermTrading(instrumentData, params, days, prevPosition):
    
    # Fitting ARIMA and predicting specified number of days
    arimaModel = ARIMA(instrumentData, order = params)
    arimaModelFit = arimaModel.fit()
    predictions = arimaModelFit.forecast(steps = days)

    # Only looking at observations within specified window
    instrumentDataWindow = instrumentData.iloc[0:days, :]
    
    # Obtaining prices and days within window with minimum and maximum days
    priceMin = [x == min(predictions) for x in predictions]
    priceMax = [x == max(predictions) for x in predictions]
    days = [x for x in range(0, days, 1)]
    dayMin = list(itertools.compress(days, priceMin))[0]
    dayMax = list(itertools.compress(days, priceMax))[0]

    # Obtaining lists of minimum and maximum days, prices, and non-minimum or maximum days
    minMaxDays = [dayMin, dayMax]
    minMaxPrice = list(instrumentDataWindow.iloc[minMaxDays, 0])
    otherDays = np.setdiff1d(days, minMaxDays)

    # Obtaining position
    position = np.full(shape = (10, 1), fill_value = prevPosition[-1]) # initial position all 0
    position[minMaxDays[0]] = -5000/minMaxPrice[0]
    position[minMaxDays[1]] = 5000/minMaxPrice[1]
    for i in otherDays: # all other days reflect previous day's position
        position[i] = position[i-1]

    return position

In [ ]:
shortTermTrading(instrumentData = dataInstrument1, params = (30, 1, 0), days = 10, prevPosition = position)

In [401]:
position = np.zeros(shape = (1,1))
i = 1
while i < 5:
    
    newPosition = shortTermTrading(instrumentData = dataInstrument1, params = (30, 1, 0), days = 10, prevPosition = position)
    position = np.concatenate((position, newPosition), axis=0)

    i = i+1

position

/Users/alanburzevski/Documents/Uni/casecomp/rho_algothon/venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'
/Users/alanburzevski/Documents/Uni/casecomp/rho_algothon/venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'
/Users/alanburzevski/Documents/Uni/casecomp/rho_algothon/venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'
/Users/alanburzevski/Documents/Uni/casecomp/rho_algothon/venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequ

array([[   0.        ],
       [   0.        ],
       [ 274.42371021],
       [ 274.42371021],
       [ 274.42371021],
       [ 274.42371021],
       [ 274.42371021],
       [ 274.42371021],
       [-271.88689505],
       [-271.88689505],
       [-271.88689505],
       [-271.88689505],
       [ 274.42371021],
       [ 274.42371021],
       [ 274.42371021],
       [ 274.42371021],
       [ 274.42371021],
       [ 274.42371021],
       [-271.88689505],
       [-271.88689505],
       [-271.88689505],
       [-271.88689505],
       [ 274.42371021],
       [ 274.42371021],
       [ 274.42371021],
       [ 274.42371021],
       [ 274.42371021],
       [ 274.42371021],
       [-271.88689505],
       [-271.88689505],
       [-271.88689505],
       [-271.88689505],
       [ 274.42371021],
       [ 274.42371021],
       [ 274.42371021],
       [ 274.42371021],
       [ 274.42371021],
       [ 274.42371021],
       [-271.88689505],
       [-271.88689505],
       [-271.88689505]])

In [ ]:
position = np.zeros(shape = (10, 1))
minmax_price = shortTermTrading(instrumentData = dataInstrument1, params = (30, 1, 0), days = 10)
position[minmax_price[0][0]] = -5000/minmax_price[1][0]
position[minmax_price[0][1]] = 5000/minmax_price[1][1]

for i in minmax_price[2]:
    position[i] = position[i-1]

position

## Modern Portfolio Theory Implementation

In [ ]:
# global constants / variables
nDays = 250
nInst = 100

In [ ]:
# Reads in first 250 days' data into numpy array givenPrices
# Input: N/A
# Output: matrix of price history from training data
def readTraining ():
    f = open("prices250.txt", "r")
    givenPrices = []
    for line in f:
        row = [float(num) for num in line.split()]
        givenPrices.append(row)
    givenPrices = np.array(givenPrices)
    # print(givenPrices.shape)
    return givenPrices

# Given the price history, output daily percentage price change matrix
# Input: givenPrices (price history)
# Output: matrix of daily percentage change; dailyReturns[i-1][j-1] = percentage change of instrument j between days i and i-1
def dailyReturns (givenPrices):
    logChange = []
    # loop through each instrument
    for inst in givenPrices.T:
        logChangeInst = np.empty((nDays, 1))
        # logChangeInst = np.zeros(nDays)
        for day in range(0, nDays - 1):
            logChangeInst[day] = np.log(inst[day + 1] / inst[day])
        logChange.append(logChangeInst)
    logChange = np.array(logChange)
    # print(logChange.T.shape)
    return logChange.T

# Given the daily price changes, output the avg. daily return, SD, variance for each instrument
def returnMeasures (dailyReturns):
    measures = np.empty((3, nInst))
    measures[0] = [np.average(inst) for inst in dailyReturns.T]
    measures[1] = [np.std(inst, ddof=1) for inst in dailyReturns.T]
    measures[2] = [np.var(inst, ddof=1) for inst in dailyReturns.T]
    return measures

# Given the price history, output the excess returns matrix
# Input: givenPrices (price history)
# Output: matrix of excess returns; excessReturns[i-1][j-1] = excess returns of instrument j on day i
def excessReturns (givenPrices):
    excessReturns = []
    # get the required matrices for further computation
    Returns = dailyReturns(givenPrices)
    measures = returnMeasures(Returns)

    for inst in range(nInst):
        instExcess = np.empty(nDays - 1) 
        instReturns = (Returns.T)[inst]
        for i in range (0, nDays - 1):
            instExcess[i] = instReturns[i] - measures[0][inst]
        excessReturns.append(instExcess)

    excessReturns = np.array(excessReturns)
    print(excessReturns.T.shape)
    return excessReturns.T

# Calculate the variance covariance matrix
# Input: excess returns
# Output: variance covariance matrix
def varCov (givenExcess):
    varCovMat = np.matmul(givenExcess.T, givenExcess)/(249 - 1) # -1 because sample std
    return varCovMat

# Calculate the scaled variance covariance matrix
# Input: variance covariance matrix
# Output: scaled variance covariance matrix
def sigma (varCovMat):
    sigmaMat = varCovMat*250
    return sigmaMat

# Calculate weights
# Input: average returns, inverse scaled variance covariance matrix, and target returns
# Output: weights for optimal portfolio given target returns
def getWeights(returns, inverseSigma, targetReturn):
    ones = np.ones(nInst)
    A = np.matmul(np.matmul(ones, inverseSigma), ones.T)
    B = np.matmul(np.matmul(ones, inverseSigma), returns.T)
    C = np.matmul(np.matmul(returns, inverseSigma), returns.T)
    delta = A * C - B**2
    lam = (C - targetReturn*B)/delta
    gam = (targetReturn*A - B)/delta

    weightsTerm1 = lam * np.matmul(inverseSigma, ones.T)
    weightsTerm2 = gam * np.matmul(inverseSigma, returns.T)
    weights = weightsTerm1 + weightsTerm2
    return(weights)  

In [ ]:
## Test to get weights

returns = returnMeasures(dailyReturns(readTraining()))[0]*250
inverseSigma = np.linalg.inv(sigma(varCov(excessReturns(readTraining()))))
targetReturn = 0.05
getWeights(returns, sigma, targetReturn)  